# Echo Analysis Tutorial

---

This interactive tutorial takes you through the steps of how to run RadxRate. RadxRate builds upon the capabilities of RadxKdp and RadxPid by calculating hourly rain rates at each radar gate depending on the local particle identification (PID) category and polarimetric values. It shares parameters files with those two applications. RadxRate will calculate the rain rates with either raw or attenuation-corrected data; the user can choose the option they prefer. The three-dimensional rain rates produced by RadxRate can be then used in RadxQpe to estimate the rain rate closest to the surface, after accounting for beam blockage, noise, and clutter.

A visual comparison of RadxKdp, RadxPid, and RadxRate and their parameter files is shown below. Each application has its own main parameter file that defines variable names and specifies the paths to the parameter file for each relevant sub-process. For example, the main RadxRate parameter file links to the specific differential phase (KDP), PID, and rate parameter files (the PID parameter file links to the PID thresholds file).

<div>
<img src="../images/radx_echo_description.png" width="600"/>
</div>

---

*Note: this tutorial is just one type of workflow to use RadxRate. There are several other workflows!*

---

## Tutorial Overview
### 1. Setup

#### Directory organization

The structure of the echo tutorial on JupyterHub is shown in the diagram below. The parent or base directory is "lrose-hub" and contains all of the notebooks, parameter files, and data for the workshop.

<div>
<img src="../images/RadxRate_structure.png" width="500"/>
</div>

#### Download raw data and prepare parameter files

Raw data files that are provided:
* gfsanl_4_2018091418.g2.tar
* KMHX20180914_191822_V06.ar2v

Parameter files (will be filled out in this tutorial):

* Grib2toMdv params
* Mdv2SoundingSpdb params
* RadxRate main params
* RadxRate Kdp_specific params
* RadxRate Pid_specific params
* RadxRate Rate_specific params
* Pid Thresholds params (S-band, simultaneous transmit)

### 2. Prepare data for analysis

* Convert Nexrad level 2 to CfRadial
    * RadxConvert
* Retrieve sounding from GFS analysis, which is needed to estimate the melting level in the PID analysis
    * Grib2toMdv, Mdv2SoundingSpdb

### 3. Run RadxRate

* Calculate KDP, run PID algorithm, estimate three-dimensional precipitation rates
    * RadxRate

### 4. Plot PID and Rate

* Visualize results of RadxRate analysis using [Py-ART](https://arm-doe.github.io/pyart/)

---

# 1. Setup
## Environment and packages

First, we import the required python packages to run this notebook. Most of the LROSE processing can be done with the os package and shell commands. At the end we will plot the output using Py-ART.

In [ ]:
import os
import pyart
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import numpy as np

Next, we set up the directory structure to simplify our commands. If you are running this notebook on the LROSE Gateway JupyterHub, these paths go to the parent directory containing all the workshop resources and the LROSE binaries. 

**If you have downloaded this notebook, please modify BASE_DIR and RAW_DIR to work on your personal machine. Note that raw data is currently provided through the Gateway. We're working to set up more permanent storage for offline users - in the meantime, contact the Gateway team to request the data.**

* BASE_DIR: the base directory containing the directories for the notebooks, data, parameter files
* RAW_DIR: the directory containing the raw data used in tutorials
* RADAR_NAME: the name of the radar used in this tutorial

In [ ]:
os.environ['BASE_DIR'] = '/home/jovyan/lrose-hub'
os.environ['RAW_DIR'] = '/home/jovyan/share/raw'
os.environ['RADAR_NAME'] = 'KMHX'
base_dir = os.environ['BASE_DIR']
radar_name = os.environ['RADAR_NAME']
!echo "Base directory: "$BASE_DIR
!echo "Radar name: "$RADAR_NAME

## Set up directories

We need to set up the required data directories. The raw radar data and GFS analysis is located within the share directory on the JupyterHub. We delete any existing files and directories specific to this tutorial to ensure we're starting with clean directories and files.

In [ ]:
## make a directory for all the data files in the echo tutorial (raw and analysis)
!rm -rf ${BASE_DIR}/data/echo_guided
!mkdir -p ${BASE_DIR}/data/echo_guided

## make subdirectory within data for the raw data
!rm -rf ${BASE_DIR}/data/echo_guided/raw
!mkdir ${BASE_DIR}/data/echo_guided/raw

## make subdirectory within data for sounding data
!rm -rf ${BASE_DIR}/data/echo_guided/sounding
!mkdir ${BASE_DIR}/data/echo_guided/sounding


We can examine the raw radar file using RadxPrint. RadxPrint reads CfRadial and other raw radar formats and prints out radar metadata, variable names, and sweep information. For example, we can look at the variables using the following command, piping the output into the bash head command.

In [ ]:
# print out the first 50 lines of RadxPrint output
!RadxPrint -f ${RAW_DIR}/echo_guided/KMHX20180914_191822_V06.ar2v | head -50


# 2. Prepare data for analysis
## 2.1 Convert Nexrad Level II to CfRadial files

We convert the raw to CfRadial using RadxConvert so the other applications can read the files. Note that in the vast majority of cases, RadxConvert can be run without a parameter file, including in this case. However, there are instances where a parameter file is useful.

Here, the *-f* flag provides the path to the raw radar file and the *-outdir* flag tells RadxConvert where the CfRadial file will be written. 

In [ ]:
!rm -rf ${BASE_DIR}/data/echo_guided/CfRadial
!mkdir ${BASE_DIR}/data/echo_guided/CfRadial
!RadxConvert -f ${RAW_DIR}/echo_guided/KMHX20180914_191822_V06.ar2v -outdir ${BASE_DIR}/data/echo_guided/CfRadial


## 2.2 Visually Inspect Data

First, let's look at a basic plot of the raw data using Py-ART.

In [ ]:
# Read CfRadial file into radar object
inDir_raw = base_dir+"/data/echo_guided/CfRadial/20180914/"
file_raw = "cfrad.20180914_191822.077_to_20180914_192511.621_KMHX_SUR.nc"
chill_raw = pyart.io.read_cfradial(inDir_raw+file_raw)

# Plot raw data

displayRaw = pyart.graph.RadarDisplay(chill_raw)
figRaw = plt.figure(1, (12, 5))

# DBZ (input)

axDbz = figRaw.add_subplot(121)
displayRaw.plot_ppi('REF', 0, vmin=-32, vmax=64.,
                    axislabels=("x(km)", "y(km)"),
                    colorbar_label="DBZ")
displayRaw.plot_range_rings([50, 100, 150])
displayRaw.plot_cross_hair(150.)
displayRaw.set_limits(xlim=(-150,150),ylim=(-150,150))

# ZDR

axZdr = figRaw.add_subplot(122)
displayRaw.plot_ppi('ZDR', 0, vmin=-1., vmax=5.,
    axislabels=("x(km)", "y(km)"),
    colorbar_label="ZDR",
    cmap="nipy_spectral")
displayRaw.plot_range_rings([50, 100, 150])
displayRaw.plot_cross_hair(150.)
displayRaw.set_limits(xlim=(-150,150),ylim=(-150,150))

## 2.3 Untar GFS analysis grib2 files for sounding
We use the GFS analysis to obtain a sounding for the PID algorithm. We need to untar the files, convert to a gridded file format (Grib2toMdv), then extract the sounding and store in in the Spdb format (Mdv2SoundingSpdb).

In [ ]:
# Grib2Mdv --> Mdv2SoundingSpdb

# make directory for grib files in data/sounding
!rm -rf ${BASE_DIR}/data/echo_guided/sounding/grib
!mkdir ${BASE_DIR}/data/echo_guided/sounding/grib

# extract file from tape archive (tar) and place in the grib directory
!tar -xvf ${RAW_DIR}/echo_guided/gfsanl_4_2018091418.g2.tar -C ${BASE_DIR}/data/echo_guided/sounding/grib/


## 2.4 Convert from Grib2 (model analysis) to MDV

The program to retrieve the model sounding requires a gridded file format called MDV. Grib2toMdv converts the GFS analysis from GRIB to MDV. Grib2toMdv needs to know where to write the gridded data files, the name of the fields, how the grid should be remapped (e.g., map projection), the size of the new grid, and the vertical levels.

| Parameter | Line # | Description |
| --- | --- | --- |
|input_dir| 61 |Optional if directory is specified in command line.|
|write_forecast| 307 |Specify if output should be written in a forecast directory structure.|
|write_non_forecast| 330 |Specify if output should be written in a non-forecast directory structure.|
|non_forecast_mdv_url| 338 |Absolute path where MDV files are written.|

<div class="alert alert-block alert-info"> <b>Task: Generate parameter file and set output type and path.</b> 
    <br>
   
1. Generate parameter file: <code lang="bash">!Grib2toMdv -print_params ${BASE_DIR}/params/echo_guided/Grib2toMdv_params</code>

2. Set <code lang="bash">write_forecast = FALSE;</code>

3. Set <code lang="bash">write_non_forecast = TRUE;</code>

4. Set <code lang="bash">non_forecast_mdv_url = "$(BASE_DIR)/data/echo/sounding/mdv";</code>

</div>
    



---

Now we list the output fields.

| Parameter | Line # | Description |
| --- | --- | --- |
|process_everything| 418 |Specify whether to process all fields, usually a subset is fine.|
|output_fields| 518 |List of output fields, where param matches GRIB files and mdv_name matches Mdv2SoundingSpdb parameter file (defaults are usually sufficient).|

<div class="alert alert-block alert-info"> <b>Task: Replace the output_fields with the list used for GFS Grib files.</b> 
    <br>
   
1. Set <code lang="bash">process_everything = FALSE;</code>

2. Replace output_fields with the list below. Replace lines 519-536, retaining lines 518 and 537 in the original parameter file. We've provided the list of variables for you.

  <code lang="bash">{
    param = "TMP",
    level = "ISBL",
    mdv_name = "TMP",
    units = KELVIN_TO_CELCIUS,
    upper_range_limit = 0,
    lower_range_limit = 0,
    encoding_type = ENCODING_FLOAT32,
    qc_default_type = BAD_VALUE,
    qc_default_value = 0,
    vert_level_min = -1,
    vert_level_max = -1,
    vert_level_dz = 1,
    use_additional_bad_data_value = FALSE,
    additional_bad_data_value = 0,
    use_additional_missing_data_value = FALSE,
    additional_missing_data_value = 0
  }
  ,
  {
    param = "RH",
    level = "ISBL",
    mdv_name = "RH",
    units = NO_CHANGE,
    upper_range_limit = 0,
    lower_range_limit = 0,
    encoding_type = ENCODING_FLOAT32,
    qc_default_type = BAD_VALUE,
    qc_default_value = 0,
    vert_level_min = -1,
    vert_level_max = -1,
    vert_level_dz = 1,
    use_additional_bad_data_value = FALSE,
    additional_bad_data_value = 0,
    use_additional_missing_data_value = FALSE,
    additional_missing_data_value = 0
  }
  ,
  {
    param = "UGRD",
    level = "ISBL",
    mdv_name = "UGRD",
    units = NO_CHANGE,
    upper_range_limit = 0,
    lower_range_limit = 0,
    encoding_type = ENCODING_FLOAT32,
    qc_default_type = BAD_VALUE,
    qc_default_value = 0,
    vert_level_min = -1,
    vert_level_max = -1,
    vert_level_dz = 1,
    use_additional_bad_data_value = FALSE,
    additional_bad_data_value = 0,
    use_additional_missing_data_value = FALSE,
    additional_missing_data_value = 0
  }
  ,
  {
    param = "VGRD",
    level = "ISBL",
    mdv_name = "VGRD",
    units = NO_CHANGE,
    upper_range_limit = 0,
    lower_range_limit = 0,
    encoding_type = ENCODING_FLOAT32,
    qc_default_type = BAD_VALUE,
    qc_default_value = 0,
    vert_level_min = -1,
    vert_level_max = -1,
    vert_level_dz = 1,
    use_additional_bad_data_value = FALSE,
    additional_bad_data_value = 0,
    use_additional_missing_data_value = FALSE,
    additional_missing_data_value = 0
  }
  ,
  {
    param = "HGT",
    level = "ISBL",
    mdv_name = "HGT",
    units = M_TO_KM,
    upper_range_limit = 0,
    lower_range_limit = 0,
    encoding_type = ENCODING_FLOAT32,
    qc_default_type = BAD_VALUE,
    qc_default_value = 0,
    vert_level_min = -1,
    vert_level_max = -1,
    vert_level_dz = 1,
    use_additional_bad_data_value = FALSE,
    additional_bad_data_value = 0,
    use_additional_missing_data_value = FALSE,
    additional_missing_data_value = 0
  }</code>

</div>

---

Finally, we specify the output grid.

| Parameter | Line # | Description |
| --- | --- | --- |
|remap_output| 672 |Specify whether to remap the output (*TRUE* or *FALSE*, usually *TRUE*).|
|out_projection_info| 708 |Set map projection type and radar location if necessary.|
|out_grid_info| 740 |User-specified domain surrounding the region of interest and radar(s). Parameters: nx/ny - number of points, minx/miny - bottom left lat/lon (deg), dx/dy - delta lon/lat (deg).|
|height_levels| 818 |User-specified height levels, optional.|

<div class="alert alert-block alert-info"> <b>Task: specify the output grid.</b> 

1. <code lang="bash">remap_output = TRUE;</code>
    
2. Set the map projection using the following values.
<code lang="bash">out_projection_info = {
    type = PROJ_LATLON,
    rotation = 0,
    origin_lat = 34.7759,
    origin_lon = -76.8762,
    ref_lat_1 = 25,
    ref_lat_2 = 25
};</code>


3. Set the grid dimensions. These values specify a 60x60 grid, with a grid spacing of 0.1º in each direction.
<code lang="bash">out_grid_info = {
    nx = 60,
    ny = 60,
    minx = -80,
    miny = 32,
    dx = 0.1,
    dy = 0.1
};</code>


</div>

---
**Now we can run Grib2toMdv to convert the files from Grib to NCAR's MDV format.**

Here, the *-params* flag directs Grib2toMdv to the parameter file and *-f* points to the raw GRIB file path. This can also be specified in the parameter *input_dir*.

In [ ]:
# Grib2 -> MDV
!Grib2toMdv -params ${BASE_DIR}/params/echo_guided/Grib2toMdv_params -f ${BASE_DIR}/data/echo_guided/sounding/grib/*.grb2


## 2.5 Convert from MDV (gridded data) to SoundingSpdb

Now that we have model analysis in a small grid around the radar(s), we can extract the sounding from the radar location and store it in a database format (Spdb). Mdv2SoundingSpdb needs the variable names in the MDV files, the geographic location of the radars, and the path where the Spdb files will be writen.

| Parameter | Line # | Description |
| --- | --- | --- |
|station_locations| 247 |Geographic location of the radar(s): radar lat, lon, altitude.|
|ouput_url| 268 |Absolute path where Spdb files are written.|

<div class="alert alert-block alert-info"> <b>Task: Generate parameter file, fill in the station location info, and set the Spdb output path.</b> 
    <br>
   
1. Generate parameter file: <code lang="bash">!Mdv2SoundingSpdb -print_params ${BASE_DIR}/params/echo_guided/Mdv2SoundingSpdb_params</code>

2. Fill in the <code lang="bash">station_locations</code> parameter for KMHX

3. Set <code lang="bash">ouput_url = "$(BASE_DIR)/data/echo/sounding/spdb";</code>


Hint: we can use RadxPrint to get the metadata and obtain the location information for the radar.

<code lang="bash">!RadxPrint -f ${BASE_DIR}/data/echo_guided/CfRadial/20180914/*.nc | head -30</code>


Below are other commands that parse the RadxPrint output for the relevant variables using bash commands grep and sed

<code lang="bash">!RadxPrint -f ${BASE_DIR}/data/echo_guided/CfRadial/20180914/*.nc | grep -i latitudeDeg | sed 's/^.*: //'</code>

<code lang="bash">!RadxPrint -f ${BASE_DIR}/data/echo_guided/CfRadial/20180914/*.nc | grep -i longitudeDeg | sed 's/^.*: //'</code>

<code lang="bash">!RadxPrint -f ${BASE_DIR}/data/echo_guided/CfRadial/20180914/*.nc | grep -i altitudeKm | sed 's/^.*: //'</code>

<code lang="bash">!RadxPrint -f ${BASE_DIR}/data/echo_guided/CfRadial/20180914/*.nc | grep -i sensorHtAglM | sed 's/^.*: //'</code>

</div>

---

Next, we'll set the field names for the variables in the MDV files we generated in the last section.

| Parameter | Line # | Description | Field Name for this Tutorial |
| --- | --- | --- | --- |
|temp_field_name| 125 |Temperature variable name.| TMP |
|rh_field_name| 145 |RH variable name.| RH |
|u_wind_field_name| 173 |U wind variable name.| UGRD |
|v_wind_field_name| 183 |V wind variable name.| VGRD |
|pressure_field_name| 205 |Pressure variable name.| Pressure |
|height_field_name| 217 |Height variable name.| HGT |

<div class="alert alert-block alert-info"> <b>Task: Set the field names to match those in the MDV files.</b> 

Set the fields names in the table above for the GFS analysis in the parameter file.

</div>

---

**Now we can run Mdv2SoundingSpdb to extract the soundings from the MDV files and place them in the Spdb database files.**

Here, the <code lang="bash">-params</code> flag directs Mdv2SoundingSpdb to the parameter file and <code lang="bash">-f</code> points to the raw GRIB file.

In [ ]:
# convert Mdv to Spdb
!Mdv2SoundingSpdb -params ${BASE_DIR}/params/echo_guided/Mdv2SoundingSpdb_params -f ${BASE_DIR}/data/echo_guided/sounding/mdv/20180914/*.mdv*
!Mdv2SoundingSpdb -params ${BASE_DIR}/params/echo_guided/Mdv2SoundingSpdb_params -f ${BASE_DIR}/data/echo_guided/sounding/mdv/20180915/*.mdv*


We can check the sounding quality using SpdbQuery, which prints out a summary of the sounding data. Here, <code lang="bash">-url</code> indicates the directory where the Spdb files are located and <code lang="bash">-start</code> and <code lang="bash">-end</code> provide the date range over which we want to inspect the sounding information. 

In [ ]:
!SpdbQuery -url ${BASE_DIR}/data/echo_guided/sounding/spdb -start "2018 09 14 00 00 00" -end " 2018 09 14 23 00 00"


# 3. Setup and run RadxRate

## RadxRate Overview

With the radar data and sounding, we can now set up and run RadxRate. First, KDP is calculated. Then polarimetric radar data, calculated KDP, and sounding data are used to run PID, which determines the dominant hydrometeor at each range gate. Finally, the 3D field of hourly rain rates are calculated from polarimetric radar and PID data.

Running RadxRate requires one main parameter file on the command line. That parameter file contains links to the KDP, PID, and precipitation rate coefficient parameter files. The RadxRate PID parameter file contains the link to the fuzzy logic PID thresholds file. We will guide you through the setup.

RadxRate requires the field names in the input CfRadial file, whether SNR and LDR are available, the method for KDP calculation, how to calculate the PID, how to calculate the precipitation rates (e.g., equations and coefficients), whether to use attenuation-correction fields, the variables to be written, and whether to censor non-weather gates. 

<div class="alert alert-block alert-info"> <b>Task: Check command line options of RadxRate</b> 
    <br>
    LROSE applications often let you link to the input files and set the output directory on the command line. Let's check that those options exist (e.g., -f and -outdir). Type the following command into the open cell below and run it!
    <br>
    <code lang="bash">!RadxRate -h</code>
</div>

## 3.1 Generate the parameter files
First, we need to generate the parameter file! LROSE applications use the <code lang="bash">-print_params</code> flag to generate parameter files with the defaults. We just need to specify the location and name of the parameter file. 

RadxRate requires 4 parameter files to run: 
* a main parameter file that sets variable names and file paths
* a KDP-specific parameter file that determines how KDP is calculated
* a PID-specific parameter file that specifies the location of the fuzzy logic thresholds
* a rate-specific parameter file that specifies the coefficients for the precipitation rate equations

<div class="alert alert-block alert-info"> <b>Task: Create the RadxRate parameter files.</b> 
<br>
Type the following commands into the open cell below and run it!
<br>
<code lang="bash">!RadxRate -print_params > ${BASE_DIR}/params/echo_guided/RadxRate_main_params</code>
    
<br>
<code lang="bash">!RadxRate -print_params_kdp > ${BASE_DIR}/params/echo_guided/RadxRate_Kdp_params</code>

<br>
<code lang="bash">!RadxRate -print_params_pid > ${BASE_DIR}/params/echo_guided/RadxRate_Pid_params</code>

<code lang="bash">!RadxRate -print_params_rate > ${BASE_DIR}/params/echo_guided/RadxRate_rate_params</code>
</div>

## 3.2 Fill out the RadxRate parameter files

Now we will step through and edit the parameter files.

**RadxRate Main Parameter file (RadxRate_main_params)**

First, we'll start with the parameters associated with the necessary file paths. Since we determined earlier that we can link to the input files and output directory on the command line, we don't need to modify them here, but we've listed the line numbers for reference.

| Parameter | Line # | Description |
| --- | --- | --- |
|input_dir| 98 |Path to input data, if not already specified on the command line.|
|KDP_params_file_path| 270 |Path to Kdp-specific parameter file.|
|PID_params_file_path| 287 |Path to PID-specific parameter file.|
|RATE_params_file_path| 328 |Path to rate-specific parameter file.|
|output_dir| 733 |Path where output files are written, if not already specified on command line with -outdir.|

<div class="alert alert-block alert-info"> <b>Task: change the kdp_params_file_path and pid_params_file_path parameters.</b> 
    <br>
    We generated the parameter files in the previous task - add those paths to the parameter file so RadxPid knows where to look.
    <ul>
<li> <code lang="bash">KDP_params_file_path = "${BASE_DIR}/params/echo_guided/RadxRate_Kdp_params"</code>
    
<li> <code lang="bash">PID_params_file_path = "${BASE_DIR}/params/echo_guided/RadxRate_Pid_params"</code>

<li> <code lang="bash">RATE_params_file_path = "${BASE_DIR}/params/echo_guided/RadxRate_rate_params"</code>
        </ul>
</div>

---

Next, we'll look at the raw file to find which variables are available and what the variable names are.

| Parameter | Line # | Description |
| --- | --- | --- |
|SNR_available| 160 |Specify if SNR is available.|
|SNR_field_name| 170 |SNR variable name.|
|DBZ_field_name| 192 |DBZ variable name.|
|ZDR_field_name| 200 |ZDR variable name.|
|PHIDP_field_name| 208 |PHIDP variable name.|
|RHOHV_field_name| 216 |RHOHV variable name.|
|LDR_available| 224 |Specify if LDR is available.|
|LDR_field_name| 232 |LDR variable name.|

<div class="alert alert-block alert-info"> <b>Task: determine if SNR and LDR are present and grab the variable names for SNR, DBZ, ZDR, PHIDP, RHOHV, and LDR (if present).</b> 
    
We can use ncdump or RadxPrint to examine the variable names in our radar files. To reduce the application output, you can also pipe information to commands like <code lang="bash">grep</code> or <code lang="bash">head</code>. 
 
<i>Use **one** of the ncdump or RadxPrint commands below in the next Jupyter cell. If you prefer to run them in a terminal window, remove the "!" symbol, which is just for Jupyter notebooks.</i>

<code lang="bash">!ncdump -h ${RAW_DIR}/data/echo_guided/cfrad*.nc </code>

<code lang="bash">!RadxPrint -f ${RAW_DIR}/data/echo_guided/cfrad*.nc</code>

</div>

Hint: NEXRAD often uses different field names than other radars.

---

RadxRate allows for use of attenuation-corrected fields in the PID and rate calculations. This is less of an issue with KMHX, which is an S-band radar, but it's good practice to account for attenutation in the PID and precipitation rates.

| Parameter | Line # | Description |
| --- | --- | --- |
|PID_use_attenuation_corrected_fields| 311 |Specify whether to use attenuation-corrected DBZ and ZDR in PID.|
|RATE_use_attenuation_corrected_fields| 352 |Specify whether to use attenuation-corrected DBZ and ZDR in the precipitation rate.|

<div class="alert alert-block alert-info"> <b>Task: use attenuation-corrected fields.</b> 
    
Set <code lang="bash">PID_use_attenuation_corrected_fields = TRUE;</code>

Set <code lang="bash">RATE_use_attenuation_corrected_fields = TRUE;</code>

</div>

---

Finally, we define the output variables and censor non-weather gates. Variables written to output files are divided into two categories: output fields (those variables created in RadxRate) and copy-through fields (variables from the input files that are copied to the output files). These variables are set in two different locations in the parameter file. 

<i>The line numbers listed here correspond to the default parameter file, but can change depending on whether additional variables are added.</i>

| Parameter | Line # | Description |
| --- | --- | --- |
|output_fields| 427 |Set which output fields are written to the output files (<code lang="bash">do_write</code>) and whether non-weather PID gates are censored (<code lang="bash">censor_non_weather</code>). The list of available output options are listed above the parameter.|
|copy_selected_input_fields_to_output| 608 |Tell RadxRate whether to copy through any fields to the output files.|
|copy_fields| 628 |Set which fields from the input files are written in the output files. <code lang="bash">input_name</code> must match the field name in the input file, <code lang="bash">output_name</code> is determined by the user (but is often the same as the original, and <code lang="bash">censor_non_weather</code> determines whether non-weather PID gates are censored for that variable.|

Of the fields calculated in RadxRate, we'll write out KDP, the PID, two rain rates, and the attenuation-corrected fields (DBZ_ATTEN_CORRECTED, ZDR_ATTEN_CORRECTED). We'll copy a few of the input fields to the output files as well. We will also censor non-weather gates.

<div class="alert alert-block alert-info"> <b>Task: specify output fields and censor non-weather gates for all variables except PID.</b> 
    
1. Output fields: write RATE_ZH, RATE_HYBRID, PID, KDP, DBZ_ATTEN_CORRECTED, and ZDR_ATTEN_CORRECTED using the <code lang="bash">do_write</code> parameter. Censor non-weather for all fields <i>**except PID**</i> using the <code lang="bash">censor_non_weather</code> parameter.

2. Set <code lang="bash">copy_selected_input_fields_to_output = TRUE;</code>
   
3. Copy-through fields: copy through REF, ZDR, and VEL from the input files (at least). Censor non-weather for all fields using the <code lang="bash">censor_non_weather</code> parameter.

4. Rename REF to DBZ to simplify plotting: set <code lang="bash">input_name = "REF"</code> and <code lang="bash">input_name = "DBZ"</code> in the copy_fields section beginning around line 628.


</div>


    
**RadxRate KDP-Specific Parameter file (RadxRate_Kdp_params)**

Usually, the default settings for KDP calculation are fine, but two commonly modified parameters are the filter length and the method for handling phase shift on backscatter. For more information on the KDP calculation in LROSE, please refer to the [LROSE Wiki](http://wiki.lrose.net/index.php/KDP_estimation).

| Parameter | Line # | Description |
| --- | --- | --- |
|KDP_fir_filter_len| 63 |Filter length used for KDP calculation.|
|KDP_psob_method| 108 |Specify method to remove phase shift on backscatter.|
          
We will use the defaults in this example, so no need to modify this parameter file.

**RadxRate PID-Specific Parameter file (RadxRate_Pid_params)**

The most important components of the PID algorithm are the PID thresholds file that sets the fuzzy logic relationships and a relevant sounding for estimating the melting level (i.e., distinguishing between liquid and frozen hydrometeors). For more information on the fuzzy logic method used in the NCAR PID algorithm, please refer to the [LROSE Wiki](http://wiki.lrose.net/index.php/RadxPid_fuzzylogic).

The PID thresholds files, particularly for S-band radars, have been tested in many environments, so those are provided for users on the LROSE wiki.

Soundings are either provided through 1) database files for each radar location and at a user-specified temporal frequency, or 2) a single sounding that is added to the PID thresholds file.

| Parameter | Line # | Description |
| --- | --- | --- |
|PID_thresholds_file_path| 25 |Path to fuzzy logic PID thresholds file.|
|PID_use_soundings_from_spdb| 259 |Specify whether soundings are in Spdb format, otherwise sounding found in fuzzy logic file.|
|PID_sounding_spdb_url| 267 |Path to Spdb soundings.|
|PID_sounding_location_name| 289 |Name of sounding location.|

We will use the Spdb sounding files that we created earlier in the tutorial.

<div class="alert alert-block alert-info"> <b>Task: download and direct RadxPid to the PID thresholds file.</b> 

1. Download the S-band PID thresholds file.

<code lang="bash">!wget http://wiki.lrose.net/images/d/de/Pid_thresholds.sband.shv.txt</code>

2. Move the file to a better directory.

<code lang="bash">!mv ./Pid_thresholds.sband.shv.txt ${BASE_DIR}/params/echo_guided/</code>

3. Add the new path of the thresholds file to the parameter file.

4. Set <code lang="bash">PID_use_soundings_from_spdb = TRUE;</code>

5. Insert the path to the soundings that we created earlier to PID_sounding_spdb_url.

6. Add the 4-letter code to the radar in this example to <code lang="bash">PID_sounding_location_name</code>.

</div>

**RadxRate rate-Specific Parameter file (RadxRate_rate_params)**

Finally, the last step is to set the coefficients for the precipitation rate equations we'll use. We've included the coefficients to use in the table below.

As we specified in the main parameter file, we will write out the precipitation rates from RATE_ZH (standard Z-R relationship) and RATE_HYBRID (NCAR's Hybrid rain rate). All precipitation rate calculations are described on the [LROSE Wiki](http://wiki.lrose.net/index.php/RadxRate_equations). The NCAR Hybrid method is described in more detail in this [AMS presentation](https://ams.confex.com/ams/37RADAR/webprogram/Paper275705.html).

| Parameter | Line # | Description | Value to Use For This Tutorial |
| --- | --- | --- | --- |
|RATE_zh_aa| 136 |R(Z) coefficients.| 0.027366 |
|RATE_zh_bb| 144 |R(Z) coefficients.| 0.69444 |
|RATE_zh_aa_snow| 160 |R(Z) coefficients in snow.| 0.0365 |
|RATE_zh_bb_snow| 168 |R(Z) coefficients in snow.| 0.625 |
|RATE_zzdr_aa| 184 |R(Z, ZDR) coefficients.| 0.0067 |
|RATE_zzdr_bb| 192 |R(Z, ZDR) coefficients.| 0.927 |
|RATE_zzdr_cc| 200 |R(Z, ZDR) coefficients.| -3.43 |
|RATE_kdp_aa| 216 |R(KDP) coefficients.| 44 |
|RATE_kdp_bb| 224 |R(KDP) coefficients.| 0.822 |
|RATE_kdpzdr_aa| 241 |R(KDP, ZDR) coefficients.| 90.8 |
|RATE_kdpzdr_bb| 249 |R(KDP, ZDR) coefficients.| 0.93 |
|RATE_kdpzdr_cc| 257 |R(KDP, ZDR) coefficients.| -2.86 |

---

**Now we can run RadxRate!**

Here, *-params* provides the link to the main RadxRate parameter file, *-f* provides the link to the files we want to process, and *-outdir* indicates where RadxRate should write the final files.

In [ ]:
!RadxRate -params ${BASE_DIR}/params/echo_guided/RadxRate_main_params -f ${BASE_DIR}/data/echo_guided/CfRadial/20180914/*.nc -outdir ${BASE_DIR}/data/echo_guided/rate


# 4. Plot PID and rate for NEXRAD Morehead City radar (KMHX)

To visualize the output in the notebook, we can use Py-ART. HawkEye is also great for visualizing data - a general parameter file can be found in the echo parameter files directory.

In [ ]:
# Read CfRadial file into radar object
inDir = base_dir+"/data/echo_guided/rate/20180914/"
file = "cfrad.20180914_191822.077_to_20180914_192511.621_KMHX_SUR.nc"
rate_kmhx = pyart.io.read_cfradial(inDir+file)
rate_kmhx.info('compact')


We can create a colormap for visualizing PID.

In [ ]:
pidmap = np.array([[0.12156862745098039, 0.46666666666666667, 0.70588235294117652, 1.0],
              [0.68235294117647061, 0.7803921568627451, 0.90980392156862744, 1.0],
              [0.59607843137254901, 0.87450980392156863, 0.54117647058823526, 1.0],
              [0.45490196078431372, 0.7686274509803922, 0.46274509803921571, 1.0],
              [0.17254901960784313, 0.62745098039215685, 0.17254901960784313, 1.0],
              [0.83921568627450982, 0.15294117647058825, 0.15686274509803921, 1.0],
              [1.0, 0.59607843137254901, 0.58823529411764708, 1.0],
              [1.0, 0.49803921568627452, 0.054901960784313725, 1.0],
              [1.0, 0.73333333333333328, 0.47058823529411764, 1.0],
              [0.61960784313725492, 0.85490196078431369, 0.89803921568627454, 1.0],
              [0.090196078431372548, 0.74509803921568629, 0.81176470588235294, 1.0],
              [0.61176470588235299, 0.61960784313725492, 0.87058823529411766, 1.0],
              [0.32156862745098042, 0.32941176470588235, 0.63921568627450975, 1.0],
              [0.859375, 0.859375, 0.859375, 1.0],
              [0.66015625, 0.66015625, 0.66015625, 1.0],
              [0.41015625, 0.41015625, 0.41015625, 1.0],
              [0.0, 0.0, 0.0, 1.0],],'f')
my_cmap2 = colors.ListedColormap(pidmap, name='ncar_pid')


In [ ]:
# Plot results of RadxRate

displayRate = pyart.graph.RadarDisplay(rate_kmhx)
figRate = plt.figure(1, (12, 10))

# DBZ (input)

axDbz = figRate.add_subplot(221)
displayRate.plot_ppi('DBZ', 0, vmin=-32, vmax=64.,
                    axislabels=("x(km)", "y(km)"),
                    colorbar_label="DBZ")
displayRate.plot_range_rings([50, 100, 150, 200, 250])
displayRate.plot_cross_hair(250.)
displayRate.set_limits(xlim=(-300,300),ylim=(-300,300))

# KDP (computed)

axKdp = figRate.add_subplot(222)
displayRate.plot_ppi('KDP', 0, vmin=0, vmax=2.,
    axislabels=("x(km)", "y(km)"),
    colorbar_label="KDP (deg/km)",
    cmap="nipy_spectral")
displayRate.plot_range_rings([50, 100, 150, 200, 250])
displayRate.plot_cross_hair(250.)
displayRate.set_limits(xlim=(-300,300),ylim=(-300,300))

# RATE_HYBRID (computed)

axHybrid = figRate.add_subplot(223)
displayRate.plot_ppi('RATE_HYBRID', 0, vmin=0, vmax=50.,
    axislabels=("x(km)", "y(km)"),
    colorbar_label="RATE_HYBRID(mm/hr)",
    cmap = "pyart_RRate11")
displayRate.plot_range_rings([50, 100, 150, 200, 250])
displayRate.plot_cross_hair(250.)
displayRate.set_limits(xlim=(-300,300),ylim=(-300,300))

# NCAR PID (computed)

axPID = figRate.add_subplot(224)
displayRate.plot_ppi('PID', 0, vmin=0.5, vmax = 17.5,
    axislabels=("x(km)", "y(km)"),
    colorbar_label="PID",
    cmap = my_cmap2, mask_outside=True)
displayRate.plot_range_rings([50, 100, 150, 200, 250])
displayRate.plot_cross_hair(250.)
displayRate.set_limits(xlim=(-300,300),ylim=(-300,300))

# plot all 17 PID categories
pid_cbar = displayRate.cbs[3]
#pid_cbar.set_ticks([1,2,3,4,5,6,7,8,9,10,11,12,13])
#pid_cbar.set_ticklabels(['cld-drops', 'drizzle', 'lt-rain', 'mod-rain', 'hvy-rain', 'hail', 'rain/hail', 'sm-hail', 'gr/rain', 'dry-snow', 'wet-snow', 'ice', 'irreg-ice'])
pid_cbar.set_ticks([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17])
pid_cbar.set_ticklabels(['cld-drops', 'drizzle', 'lt-rain', 'mod-rain', 'hvy-rain', 'hail', 'rain/hail', 'sm-hail', 'gr/rain', 'dry-snow', 'wet-snow', 'ice', 'irreg-ice', 'slw', 'insects', '2nd-trip', 'clutter'])

figRate.tight_layout()

plt.show()


Next, we'll remake the same plot with the attenuation-corrected DBZ field.

In [ ]:
# Plot results of RadxRate, 

displayRate = pyart.graph.RadarDisplay(rate_kmhx)
figRate = plt.figure(1, (12, 10))

# DBZ (input)

axDbz = figRate.add_subplot(221)
displayRate.plot_ppi('DBZ_ATTEN_CORRECTED', 0, vmin=-32, vmax=64.,
                    axislabels=("x(km)", "y(km)"),
                    colorbar_label="Attenuation-Corrected DBZ",cmap="pyart_HomeyerRainbow")
displayRate.plot_range_rings([50, 100, 150, 200, 250])
displayRate.plot_cross_hair(250.)
displayRate.set_limits(xlim=(-300,300),ylim=(-300,300))

# KDP (computed)

axKdp = figRate.add_subplot(222)
displayRate.plot_ppi('KDP', 0, vmin=0, vmax=2.,
    axislabels=("x(km)", "y(km)"),
    colorbar_label="KDP (deg/km)",
    cmap="nipy_spectral")
displayRate.plot_range_rings([50, 100, 150, 200, 250])
displayRate.plot_cross_hair(250.)
displayRate.set_limits(xlim=(-300,300),ylim=(-300,300))

# RATE_HYBRID (computed)

axHybrid = figRate.add_subplot(223)
displayRate.plot_ppi('RATE_HYBRID', 0, vmin=0, vmax=50.,
    axislabels=("x(km)", "y(km)"),
    colorbar_label="RATE_HYBRID(mm/hr)",
    cmap = "pyart_RRate11")
displayRate.plot_range_rings([50, 100, 150, 200, 250])
displayRate.plot_cross_hair(250.)
displayRate.set_limits(xlim=(-300,300),ylim=(-300,300))

# NCAR PID (computed)

axPID = figRate.add_subplot(224)
displayRate.plot_ppi('PID', 0, vmin=0.5, vmax = 17.5,
    axislabels=("x(km)", "y(km)"),
    colorbar_label="PID",
    cmap = my_cmap2, mask_outside=True)
displayRate.plot_range_rings([50, 100, 150, 200, 250])
displayRate.plot_cross_hair(250.)
displayRate.set_limits(xlim=(-300,300),ylim=(-300,300))

# plot all 17 PID categories
pid_cbar = displayRate.cbs[3]
#pid_cbar.set_ticks([1,2,3,4,5,6,7,8,9,10,11,12,13])
#pid_cbar.set_ticklabels(['cld-drops', 'drizzle', 'lt-rain', 'mod-rain', 'hvy-rain', 'hail', 'rain/hail', 'sm-hail', 'gr/rain', 'dry-snow', 'wet-snow', 'ice', 'irreg-ice'])
pid_cbar.set_ticks([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17])
pid_cbar.set_ticklabels(['cld-drops', 'drizzle', 'lt-rain', 'mod-rain', 'hvy-rain', 'hail', 'rain/hail', 'sm-hail', 'gr/rain', 'dry-snow', 'wet-snow', 'ice', 'irreg-ice', 'slw', 'insects', '2nd-trip', 'clutter'])

figRate.tight_layout()

plt.show()


Finally, we can compare the rain rates calculated from a standard Z-R relationship and NCAR's Hybrid method.

In [ ]:
# Plot results of RadxRate 

displayRate = pyart.graph.RadarDisplay(rate_kmhx)
figRate = plt.figure(1, (12, 5))

# RATE_ZH (computer)

axDbz = figRate.add_subplot(121)
displayRate.plot_ppi('RATE_ZH', 0, vmin=0, vmax=50.,
    axislabels=("x(km)", "y(km)"),
    colorbar_label="RATE_HYBRID(mm/hr)",
    cmap = "pyart_RRate11")
displayRate.plot_range_rings([50, 100, 150, 200, 250])
displayRate.plot_cross_hair(250.)
displayRate.set_limits(xlim=(-300,300),ylim=(-300,300))

# RATE_HYBRID (computed)

axHybrid = figRate.add_subplot(122)
displayRate.plot_ppi('RATE_HYBRID', 0, vmin=0, vmax=50.,
    axislabels=("x(km)", "y(km)"),
    colorbar_label="RATE_HYBRID(mm/hr)",
    cmap = "pyart_RRate11")
displayRate.plot_range_rings([50, 100, 150, 200, 250])
displayRate.plot_cross_hair(250.)
displayRate.set_limits(xlim=(-300,300),ylim=(-300,300))

figRate.tight_layout()

plt.show()


